In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import os
from google.colab import drive

os.chdir("drive/My Drive/IIITH/Advanced_NLP//Assignment_2")

In [4]:
eng_hi_Corpus = open("joint_Corpus.txt","r", encoding='utf-8')
eng_hi_Corpus = eng_hi_Corpus.read().splitlines()
df = pd.DataFrame(eng_hi_Corpus)
X_train, X_test = train_test_split(df[0:10000], test_size=0.1, random_state=0)

In [5]:
eng_hi_train = np.array(X_train)
eng_hi_test = np.array(X_test)

### create train file for English and Hindi by splitting the text by "!"
eng_Corpus_train = open("eng_Corpus_train.txt","w", encoding='utf-8')
hin_Corpus_train = open("hin_Corpus_train.txt","w", encoding='utf-8')

for i in range(eng_hi_train.shape[0]):
    eng_Corpus_train.write(eng_hi_train[i][0].split('|')[0] + '\n')
    hin_Corpus_train.write(eng_hi_train[i][0].split('|')[1] + '\n')
  
eng_Corpus_train.close()
hin_Corpus_train.close() 

### create test file for English and Hindi by splitting the text by "!"

eng_Corpus_test = open("eng_Corpus_test.txt","w", encoding='utf-8')
hin_Corpus_test = open("hin_Corpus_test.txt","w", encoding='utf-8')

for i in range(eng_hi_test.shape[0]):
    eng_Corpus_test.write(eng_hi_test[i][0].split('|')[0] + '\n')
    hin_Corpus_test.write(eng_hi_test[i][0].split('|')[1] + '\n')
  
eng_Corpus_test.close()
hin_Corpus_test.close() 

In [6]:
from keras.models import Model
from keras.models import load_model,save_model
from keras.layers import Input, LSTM, GRU, Dense, Embedding, Activation, dot, concatenate, Bidirectional, Attention
from keras.preprocessing.text import Tokenizer
from keras.optimizers import RMSprop
from keras.callbacks import EarlyStopping
import tensorflow_datasets as tfds
from keras.layers.wrappers import TimeDistributed, Bidirectional
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import numpy as np
import nltk

In [7]:
BATCH_SIZE = 32
EPOCHS = 20
LSTM_NODES =128
NUM_SENTENCES = 9000
MAX_NUM_WORDS = 10000
EMBEDDING_SIZE = 100

In [8]:
input_sentences = []
output_sentences = []
output_sentences_inputs = []

En_Corpus = open('eng_Corpus_train.txt','r',encoding = 'utf-8')
cnt = 0 
for line in En_Corpus.readlines():
    #print(line)
    if cnt < NUM_SENTENCES:
      input_sentences.append(line.strip('\n'))
    cnt = cnt+1
En_Corpus.close()

Hi_Corpus = open('hin_Corpus_train.txt','r',encoding = 'utf-8')
cnt = 0 
for line in Hi_Corpus.readlines():
  if cnt < NUM_SENTENCES:
    output_sentences.append(line.strip('\n') + ' <EOS>')
    output_sentences_inputs.append('<SOS> '+line.strip('\n') )
  cnt = cnt +1
Hi_Corpus.close()

print("num samples input:", len(input_sentences))
print("num samples output:", len(output_sentences))

num samples input: 9000
num samples output: 9000


In [9]:
input_tokenizer = Tokenizer(filters='')
input_tokenizer.fit_on_texts(input_sentences)
input_integer_seq = input_tokenizer.texts_to_sequences(input_sentences)

word2idx_inputs = input_tokenizer.word_index
print('Total unique words in the input: %s' % len(word2idx_inputs))

max_input_len = max(len(sen) for sen in input_integer_seq)
print("Length of longest sentence in input: %g" % max_input_len)
num_words_input = len(word2idx_inputs) + 1

Total unique words in the input: 8219
Length of longest sentence in input: 5


In [10]:
output_tokenizer = Tokenizer(filters='')
output_tokenizer.fit_on_texts(output_sentences + output_sentences_inputs)
output_integer_seq = output_tokenizer.texts_to_sequences(output_sentences)
output_input_integer_seq = output_tokenizer.texts_to_sequences(output_sentences)

word2idx_outputs = output_tokenizer.word_index
print('Total unique words in the output: %s' % len(word2idx_outputs))

num_words_output = len(word2idx_outputs) + 1
max_out_len = max(len(sen) for sen in output_integer_seq)
print("Length of longest sentence in the output: %g" % max_out_len)

Total unique words in the output: 8270
Length of longest sentence in the output: 29


In [11]:
input_text = input_sentences
target_text_input = output_sentences_inputs
target_text_output = output_sentences

input_integer_seq = input_tokenizer.texts_to_sequences(input_text)
encoder_input_sequences = pad_sequences(input_integer_seq, maxlen=max_input_len)
         
output_input_integer_seq = output_tokenizer.texts_to_sequences(target_text_input)
decoder_input_sequences = pad_sequences(output_input_integer_seq, maxlen=max_out_len, padding='post')
          
output_integer_seq = output_tokenizer.texts_to_sequences(target_text_output)
decoder_output_sequences = pad_sequences(output_integer_seq, maxlen=max_out_len, padding='post')
          
decoder_targets_one_hot = np.zeros((len(input_sentences),max_out_len,num_words_output),dtype='float32')

#print('decoder_targets_one_hot ',decoder_targets_one_hot.shape)


for i, d in enumerate(decoder_output_sequences):
  for t, word in enumerate(d):
    decoder_targets_one_hot[i, t, word] = 1
          

In [12]:
model = load_model('Seq2Seq_assignment2_v1.h5')
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 5)]          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 29)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 5, 128)       1052160     input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 29, 128)      1058688     input_2[0][0]                    
_______________________________________________________________________________________

In [13]:
#For Inference Purpose
encoder_inputs_placeholder = Input(shape=(max_input_len,))#model.get_layer('input_1')
encoder_embedding = model.get_layer('embedding')
encoder_lstm = model.get_layer('lstm')

x = encoder_embedding(encoder_inputs_placeholder)
encoder_outputs, h, c = encoder_lstm(x)
encoder_states = [h, c]

#Define Encoder Model
encoder_model = Model(encoder_inputs_placeholder, encoder_states)  #without attention

In [14]:
decoder_state_input_h = Input(shape=(LSTM_NODES,),name='decoder_h_input')
decoder_state_input_c = Input(shape=(LSTM_NODES,),name='decoder_c_input')
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_embedding = model.get_layer('embedding_1')

decoder_inputs_single = Input(shape=(1,),name='decoder_single_input')
decoder_inputs_single_x = decoder_embedding(decoder_inputs_single)
decoder_lstm = model.get_layer('lstm_1')

decoder_outputs, h, c = decoder_lstm(decoder_inputs_single_x, initial_state=decoder_states_inputs)
decoder_states = [h, c]
print('decoder_outputs shape:',decoder_outputs.shape)

decoder_dense = model.get_layer('dense')

decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model(
    [decoder_inputs_single] + decoder_states_inputs,
    [decoder_outputs] + decoder_states
)

decoder_outputs shape: (None, 1, 128)


In [15]:
idx2word_input = {v:k for k, v in word2idx_inputs.items()}
idx2word_target = {v:k for k, v in word2idx_outputs.items()}

In [16]:
def translate_sentence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = word2idx_outputs['<sos>']
    eos = word2idx_outputs['<eos>']
    output_sentence = []

    for _ in range(max_out_len):
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        idx = np.argmax(output_tokens[0, 0, :])

        if eos == idx:
            break

        word = ''

        if idx > 0:
            word = idx2word_target[idx]
            output_sentence.append(word)

        target_seq[0, 0] = idx
        states_value = [h, c]

    return ' '.join(output_sentence)

In [17]:
# prediction on test data

input_sentences_test = []
output_sentences_test = []

En_Corpus_test = open('eng_Corpus_test.txt','r',encoding = 'utf-8')
cnt = 0 
for line in En_Corpus_test.readlines():
    #print(line)
    if cnt < NUM_SENTENCES:
      input_sentences_test.append(line.strip('\n'))
    cnt = cnt+1
En_Corpus_test.close()

Hi_Corpus_test = open('hin_Corpus_test.txt','r',encoding = 'utf-8')
cnt = 0 
for line in Hi_Corpus_test.readlines():
  if cnt < NUM_SENTENCES:
    output_sentences_test.append(line.strip('\n') + ' <EOS>')
  cnt = cnt +1
Hi_Corpus_test.close()


In [18]:
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction
smooth = SmoothingFunction().method4
def calculate_bleu(text1,text2):
  hypothesis = text1.split()
  reference = text2.split()

  reference_list = [reference] # list of references for 1 sentence.
  bleu_score = nltk.translate.bleu_score.sentence_bleu(reference_list, hypothesis,smoothing_function = smooth)
  return bleu_score

In [19]:
test_pred_result = open('test_prediction_without_attention','w',encoding = 'utf-8')
test_sent_bleu = {}
for s in range(len(input_sentences_test)):
  input_integer_seq = input_tokenizer.texts_to_sequences([input_sentences_test[s]])
  encoder_input_sequences = pad_sequences(input_integer_seq, maxlen=max_input_len)
  translation = translate_sentence(encoder_input_sequences)
  sent_bleu = calculate_bleu(output_sentences_test[s].strip('<EOS>'),translation)
  test_sent_bleu[s] = sent_bleu
  text = str(s) + ' ' + input_sentences_test[s] + '|' + output_sentences_test[s].strip('<EOS>') + 'Predicted: ' + translation + ' Bleu Score: ' + str(sent_bleu)
  test_pred_result.write(text + '\n')
test_pred_result.close()

test_cospus_score = np.sum(list(test_sent_bleu.values()))/len(input_sentences_test)
print('Test Corpus Bleu Score without attention:',test_cospus_score)

Test Corpus Bleu Score without attention: 0.1549682070309914


In [20]:
test_sent_bleu_sorted_index = sorted(test_sent_bleu, key=test_sent_bleu.get, reverse=True)

test_best = []
for i in test_sent_bleu_sorted_index[0:20]:
  test_best.append(i)

for s in test_best:
  input_integer_seq = input_tokenizer.texts_to_sequences([input_sentences_test[s]])
  encoder_input_sequences = pad_sequences(input_integer_seq, maxlen=max_input_len)
  translation = translate_sentence(encoder_input_sequences)
  sent_bleu = calculate_bleu(output_sentences_test[s].strip('<EOS>'),translation)
  test_sent_bleu[s] = sent_bleu
  text = str(s) + ' ' + input_sentences_test[s] + '|' + output_sentences_test[s].strip('<EOS>') + 'Predicted: ' + translation + ' Bleu Score: ' + str(sent_bleu)
  print(text)

133 "Sweetheart I swear on you."|"जानेमन मैं तुम पर कसम खाता हूँ।" Predicted: "जानेमन मैं तुम पर कसम खाता हूँ।" Bleu Score: 1.0
134 "Sweetheart I swear on you."|"जानेमन मैं तुम पर कसम खाता हूँ।" Predicted: "जानेमन मैं तुम पर कसम खाता हूँ।" Bleu Score: 1.0
179 What's going on here?|यहाँ क्या हो रहा है? Predicted: यहाँ क्या हो रहा है? Bleu Score: 1.0
247 Get off of me.|मुझसे दूर हो जाओ. Predicted: मुझसे दूर हो जाओ. Bleu Score: 1.0
253 man speaking native language:|पुरुष अपनी मातृभाषा में बोल रहा है: Predicted: पुरुष अपनी मातृभाषा में बोल रहा है: Bleu Score: 1.0
301 "Sweetheart I swear on you."|"जानेमन मैं तुम पर कसम खाता हूँ।" Predicted: "जानेमन मैं तुम पर कसम खाता हूँ।" Bleu Score: 1.0
326 What are you doing?|तुम क्या कर रहे हो? Predicted: तुम क्या कर रहे हो? Bleu Score: 1.0
507 "Sweetheart I swear on you."|"जानेमन मैं तुम पर कसम खाता हूँ।" Predicted: "जानेमन मैं तुम पर कसम खाता हूँ।" Bleu Score: 1.0
534 Is she all right?|वह सब ठीक है? Predicted: वह सब ठीक है? Bleu Score: 1.0
561 What a